In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle

from xgboost import XGBRegressor
from IPython.display import clear_output

import itertools
import tensorflow
from tensorflow.python.keras.layers import Input, Dense
from tensorflow.python.keras.models import Model
import time
from sklearn.gaussian_process import GaussianProcessRegressor

In [50]:
### read all the simulation data with different properties
df1 = pd.read_csv("Data/FE_constAng3_29param.csv")

df2 = pd.read_csv("Data/FE_Var_SizeAngle3_29param.csv")

df3 = pd.read_csv("Data/FE_Var_SizeConstAng3_29param.csv")

df4 = pd.read_csv("Data/FE_varAng3_29param.csv")

# df5 = pd.read_csv("Data/FE_constAng5_29param.csv")

# df6 = pd.read_csv("Data/FE_varAng5_29param.csv")

# df7 = pd.read_csv("Data/FE_constAng7_29param.csv")

# df8 = pd.read_csv("Data/FE_varAng7_29param.csv")


### As there are 15 repeated cases it makes the resullts better,
### wich might not be case, unless we group them touse the mean value of different trials
df9 = pd.read_csv("Data/FE_constAng3_repeat_29param.csv")

df10 = pd.read_csv("Data/FE_repeatRound2_29param.csv") 


##############################################
df_total = pd.concat([df1, df2, df3, df4, 
#                       df5, df6, df7, df8 ,
                      df9 , 
#                       df10
                       ], axis = 0, ignore_index = True)

df = df_total.groupby(['Number_pieces', 'Length_ratio','angle1_9p', 'angle2_9', 'angle3_9p', 'angle4_9p'
                               ,'angle1_25p', 'angle2_25p', 'angle3_25p',
                               'angle4_25p','angle5_25p', 'angle6_25p','angle1_49p', 'angle2_49p',
                               'angle3_49p','angle4_49p','angle5_49p', 'angle6_49p', 'angle7_49p',
                               'angle8_49p'
                              ], as_index=False).mean()

df = shuffle(df)
df.shape

(74, 29)

In [55]:
print(
    df.loc[(df.FricDissipRate > 4)].index[0], 
    df.loc[(df.Safety_factor < 0.015)].index[0], 
#       df.loc[(df.Tot_contactEngy > 50)].index[0]
)

48 2


In [56]:
df_filtered = df.drop([
    df.loc[(df.Safety_factor < 0.015)].index[0],
                      df.loc[(df.FricDissipRate > 4)].index[0],
#                       df.loc[(df.Tot_contactEngy > 50)].index[0]
                      ], axis = 0)
df = df_filtered.reset_index(drop=True)

In [57]:
# ###------- normalizing data 
normalizer = MinMaxScaler()
df_norm = normalizer.fit_transform(df)

df = pd.DataFrame(df_norm, columns=df.columns )
# print(df.keys())

In [58]:
# df_clean3 = df.drop(df.iloc[:,6:20], axis = 'columns')
X = df.iloc[:, 0:6]



## We define all the models in functions to be able to call them multiple times and for different outputs. 
**The hyperparameters for each output has been determined through grid search in another notebook over all the input parameters for 3 by 3 pannel**

In [59]:
def Hyperparam(output):
    if output == 'Safety_factor' :
        Y1 = df.iloc[:, 20:21]
        hyperparams = [0.1, 0.1 , 3, 100, 0.635]
    elif output == 'Oop_deform' : 
        Y1 = df.iloc[:, 21:22] 
        hyperparams = [1, 0.5, 3, 20, 0.632]
    elif output == 'Tot_contactEngy' : 
        Y1 = df.iloc[:, 22:23]
        hyperparams = [1, 0.5, 3, 20, 0.632]
    elif output == 'Elast_strainEngy':
        Y1 = df.iloc[:, 23:24] 
        hyperparams = [0.5, 0.1, 7, 50, 0.632]
    elif output == 'Edge_temp':
        Y1 = df.iloc[:, 24:25]
        hyperparams = [0.1, 0.5, 3, 50, 0.632]
    elif output == 'Avr_frictForce':
        Y1 = df.iloc[:, 25:26]
        hyperparams = [0.5, 0.5, 3, 100, 1]
    elif output == 'HeatRate':
        Y1 = df.iloc[:, 26:27]
        hyperparams = [0.5, 0.5, 3, 100, 1]
    elif output == 'IntEngy':
        Y1 = df.iloc[:, 27:28]
        hyperparams = [1, 0.1, 3, 100, 0.632]
    elif output == 'FricDissipRate':
        Y1 = df.iloc[:, 28:29]
        hyperparams = [0.5, 0.1, 5, 50, 1]
    return Y1, hyperparams

In [60]:
""" Define different models"""

def MODELS(model_name, output):
    Y1, hyperparams = Hyperparam(output)
    if model_name == 'model1' : 
        model = Pipeline([
            ('linear_regression', LinearRegression())
        ])
    elif model_name == 'model2' :
        model = Pipeline([   # higher degree might causes overfitting
            ('poly', PolynomialFeatures(degree=2, include_bias=False)), 
            ('linear_regression', LinearRegression())
        ])

    elif model_name == 'model3' :
        model = Pipeline([   
            ('poly', PolynomialFeatures(degree=3, include_bias=False)), 
            ('linear_regression', LinearRegression())
        ])
        
    elif model_name == 'model4' :
        model = Pipeline([   
            ('poly', PolynomialFeatures(degree=4, include_bias=False)), 
            ('linear_regression', LinearRegression())
        ])

    elif model_name == 'model5' :
        ## XGB on original standardized inputs
        model = Pipeline([
            ('XGB', XGBRegressor(max_depth=hyperparams[2],                 # Depth of each tree
                            learning_rate=hyperparams[1],            # How much to shrink error in each subsequent training. Trade-off with no. estimators.
                            n_estimators=hyperparams[3],             # How many trees to use, the more the better, but decrease learning rate if many used.
                            verbosity=1,                  # If to show more errors or not.
                            objective='reg:squarederror',  # Type of target variable. for classifieer use 'binary:logistic'
                            n_jobs=4,                     # Parallel jobs to run. Set your processor number.
                            gamma=0.001,                  # Minimum loss reduction required to make a further partition on a leaf node of the tree. (Controls growth!)
                            subsample=hyperparams[4],              # Subsample ratio. Can set lower than 1. If we want perfect boosting
                            colsample_bytree=1,           # Subsample ratio of columns when constructing each tree.
                            colsample_bylevel=1,          # Subsample ratio of columns when constructing each level. 0.33 is similar to random forest. sqrt(no.var)
                            colsample_bynode=1,           # Subsample ratio of columns when constructing each split.
                            scale_pos_weight=0.5,           # Balancing of positive and negative weights.
                            base_score=hyperparams[0],               # Global bias. Set to average of the target rate.
                            random_state=20210614,        # Seed
                            missing=1                  # How are nulls encoded?
                            ))
        ])

    elif model_name == 'model6' :
        ### XGB on 2nd order polynomial of inputs
        model = Pipeline([
            ('poly', PolynomialFeatures(degree=2, include_bias=False)),
            ('XGB', XGBRegressor(max_depth=hyperparams[2],                 # Depth of each tree
                            learning_rate=hyperparams[1],            # How much to shrink error in each subsequent training. Trade-off with no. estimators.
                            n_estimators=hyperparams[3],             # How many trees to use, the more the better, but decrease learning rate if many used.
                            verbosity=1,                  # If to show more errors or not.
                            objective='reg:squarederror',  # Type of target variable. for classifieer use 'binary:logistic'
                            n_jobs=4,                     # Parallel jobs to run. Set your processor number.
                            gamma=0.001,                  # Minimum loss reduction required to make a further partition on a leaf node of the tree. (Controls growth!)
                            subsample=hyperparams[4],              # Subsample ratio. Can set lower than 1. If we want perfect boosting
                            colsample_bytree=1,           # Subsample ratio of columns when constructing each tree.
                            colsample_bylevel=1,          # Subsample ratio of columns when constructing each level. 0.33 is similar to random forest. sqrt(no.var)
                            colsample_bynode=1,           # Subsample ratio of columns when constructing each split.
                            scale_pos_weight=0.5,           # Balancing of positive and negative weights.
                            base_score=hyperparams[0],               # Global bias. Set to average of the target rate.
                            random_state=20210614,        # Seed
                            missing=1                  # How are nulls encoded?
                            ))
        ])

    elif model_name == 'model7' :
        ### XGB on 3rd order polynomial of inputs
        model = Pipeline([
            ('poly', PolynomialFeatures(degree=3, include_bias=False)),
            ('XGB', XGBRegressor(max_depth=hyperparams[2],                 # Depth of each tree
                            learning_rate=hyperparams[1],            # How much to shrink error in each subsequent training. Trade-off with no. estimators.
                            n_estimators=hyperparams[3],             # How many trees to use, the more the better, but decrease learning rate if many used.
                            verbosity=1,                  # If to show more errors or not.
                            objective='reg:squarederror',  # Type of target variable. for classifieer use 'binary:logistic'
                            n_jobs=4,                     # Parallel jobs to run. Set your processor number.
                            gamma=0.001,                  # Minimum loss reduction required to make a further partition on a leaf node of the tree. (Controls growth!)
                            subsample=hyperparams[4],              # Subsample ratio. Can set lower than 1. If we want perfect boosting
                            colsample_bytree=1,           # Subsample ratio of columns when constructing each tree.
                            colsample_bylevel=1,          # Subsample ratio of columns when constructing each level. 0.33 is similar to random forest. sqrt(no.var)
                            colsample_bynode=1,           # Subsample ratio of columns when constructing each split.
                            scale_pos_weight=0.5,           # Balancing of positive and negative weights.
                            base_score=hyperparams[0],               # Global bias. Set to average of the target rate.
                            random_state=20210614,        # Seed
                            missing=1                  # How are nulls encoded?
                            ))
        ])
    return model

In [61]:
"""Cross validation error on all the data"""

def cv_loss(Xtrain, Ytrain, output):
    kf = KFold(n_splits=10)
    print('Train cross validation loss on ',str(output),'for different models:')

    #################################################
    print('\nLinear regression:')
    print((-cross_val_score(MODELS('model1', output), Xtrain, Ytrain,
                          cv=kf, scoring='neg_mean_squared_error').mean()).round(6)*100,'%')

    #####################

    print('\n2nd order Polynomial + LR:')
    print((-cross_val_score(MODELS('model2', output), Xtrain, Ytrain,
                          cv=kf, scoring='neg_mean_squared_error').mean()).round(6)*100,'%')

    ########################    

    print('\n3rd order Polynomial + LR:')
    print((-cross_val_score(MODELS('model3', output), Xtrain, Ytrain,
                          cv=kf, scoring='neg_mean_squared_error').mean()).round(6)*100,'%')

    ########################

    print('\n4th order Polynomial + LR:')
    print((-cross_val_score(MODELS('model4', output), Xtrain, Ytrain,
                          cv=kf, scoring='neg_mean_squared_error').mean()).round(6)*100,'%')

    ########################

    print('\nXGB:')
    print((-cross_val_score(MODELS('model5', output), Xtrain, Ytrain,
                            cv=kf, scoring='neg_mean_squared_error').mean()).round(6)*100,'%')

    ##################

    print('\n2nd order Polynomial + XGB:')
    print((-cross_val_score(MODELS('model6', output), Xtrain, Ytrain,
                            cv=kf, scoring='neg_mean_squared_error').mean()).round(6)*100,'%')

    ##################

    print('\n3rd order Polynomial + XGB:')
    print((-cross_val_score(MODELS('model7', output), Xtrain, Ytrain,
                            cv=kf, scoring='neg_mean_squared_error').mean()).round(6)*100,'%')

## Comparing cross-validation error of different models on all the data

In [62]:
OUTPUTS = ['Safety_factor', 'Oop_deform', 'Tot_contactEngy', 'Elast_strainEngy',
       'Edge_temp', 'Avr_frictForce', 'HeatRate', 'IntEngy', 'FricDissipRate']

for output in OUTPUTS:
    Y1, hyperparams = Hyperparam(output)
    cv_loss(X, Y1, output)
    print('***********************************************************')

Train cross validation loss on  Safety_factor for different models:

Linear regression:
4.3442 %

2nd order Polynomial + LR:
3.3484 %

3rd order Polynomial + LR:
5.113 %

4th order Polynomial + LR:
14.671999999999999 %

XGB:
1.9375 %

2nd order Polynomial + XGB:
1.8374000000000001 %

3rd order Polynomial + XGB:
1.6923000000000001 %
***********************************************************
Train cross validation loss on  Oop_deform for different models:

Linear regression:
3.2368 %

2nd order Polynomial + LR:
2.4147 %

3rd order Polynomial + LR:
2.3581000000000003 %

4th order Polynomial + LR:
9.7469 %

XGB:
2.0493 %

2nd order Polynomial + XGB:
1.8469 %

3rd order Polynomial + XGB:
1.8619 %
***********************************************************
Train cross validation loss on  Tot_contactEngy for different models:

Linear regression:
4.5172 %

2nd order Polynomial + LR:
2.75 %

3rd order Polynomial + LR:
7.504700000000001 %

4th order Polynomial + LR:
15.5728 %

XGB:
3.0177 %

2n

In [63]:
def modelPerformance(X, Y1):
    losses = []
    R2es = []

    models = {'LR': MODELS('model1', output), 
              '2poly_LR': MODELS('model2', output), '3poly_LR': MODELS('model3', output), #'4poly_LR': model4,
              'XGB': MODELS('model5', output), '2poly_XGB': MODELS('model6', output), 
              '3poly_XGB': MODELS('model7', output),}

    n = 200   # how many different test-train splits
    for key, value in models.items():

        losses = []
        R2es = []
        t = time.process_time()

        for i in range(n):
            X_train, X_test,Y_train, Y_test, = train_test_split(X, Y1 , test_size = 0.20)
            Models = value.fit(X_train, Y_train)
            predict = Models.predict(X_test)
            loss = mean_squared_error(Y_test, predict)
            R2 = r2_score(Y_test, predict)
            losses.append(loss)
            R2es.append(R2)

        elapsed_time = time.process_time() - t
        print('Avr. test loss on ',str(output),'for different models:')
        print('Avr. MSE for: ',key ,' = ', (np.mean(np.array(losses))*100).round(6))
        print('Avr. R2 for: ',key ,' = ', (np.mean(np.array(R2es))*100).round(6),' %')
        print('time for each split: ',round(elapsed_time/n, 6), 's \n*********\n')

## Comparing performance of different models for 100 different train-test splits

In [64]:
OUTPUTS = ['Safety_factor', 'Oop_deform', 'Tot_contactEngy', 'Elast_strainEngy',
       'Edge_temp', 'Avr_frictForce', 'HeatRate', 'IntEngy', 'FricDissipRate']

for output in OUTPUTS:
    Y1, hyperparams = Hyperparam(output)
    modelPerformance(X, Y1)
    print('***********************************************************')

Avr. test loss on  Safety_factor for different models:
Avr. MSE for:  LR  =  4.427168
Avr. R2 for:  LR  =  -18.370473  %
time for each split:  0.005598 s 
*********

Avr. test loss on  Safety_factor for different models:
Avr. MSE for:  2poly_LR  =  3.622436
Avr. R2 for:  2poly_LR  =  -0.35674  %
time for each split:  0.006633 s 
*********

Avr. test loss on  Safety_factor for different models:
Avr. MSE for:  3poly_LR  =  5.653789
Avr. R2 for:  3poly_LR  =  -62.158482  %
time for each split:  0.013495 s 
*********

Avr. test loss on  Safety_factor for different models:
Avr. MSE for:  XGB  =  1.964528
Avr. R2 for:  XGB  =  43.690469  %
time for each split:  0.096944 s 
*********

Avr. test loss on  Safety_factor for different models:
Avr. MSE for:  2poly_XGB  =  1.951274
Avr. R2 for:  2poly_XGB  =  41.763368  %
time for each split:  0.115337 s 
*********

Avr. test loss on  Safety_factor for different models:
Avr. MSE for:  3poly_XGB  =  1.753036
Avr. R2 for:  3poly_XGB  =  51.849246  %


Avr. test loss on  IntEngy for different models:
Avr. MSE for:  2poly_XGB  =  1.09995
Avr. R2 for:  2poly_XGB  =  75.878413  %
time for each split:  0.11363 s 
*********

Avr. test loss on  IntEngy for different models:
Avr. MSE for:  3poly_XGB  =  1.081394
Avr. R2 for:  3poly_XGB  =  76.570347  %
time for each split:  0.134941 s 
*********

***********************************************************
Avr. test loss on  FricDissipRate for different models:
Avr. MSE for:  LR  =  3.561119
Avr. R2 for:  LR  =  43.374276  %
time for each split:  0.008176 s 
*********

Avr. test loss on  FricDissipRate for different models:
Avr. MSE for:  2poly_LR  =  3.309781
Avr. R2 for:  2poly_LR  =  45.253246  %
time for each split:  0.006653 s 
*********

Avr. test loss on  FricDissipRate for different models:
Avr. MSE for:  3poly_LR  =  4.688898
Avr. R2 for:  3poly_LR  =  21.91281  %
time for each split:  0.01366 s 
*********

Avr. test loss on  FricDissipRate for different models:
Avr. MSE for:  XGB  

## Checking NN results

In [22]:
def NN_MultiLayer(Xtrain, Y_train, Xtest, Y_test):
#     tensorflow.random.set_seed(20210614)   # to fix initial coeficients 
    input = Input(shape=(6,))
    x = Dense(64, activation='relu')(input)
    x = Dense(32, activation='relu')(x)
    output = Dense(1)(x)
    model = Model(inputs=input, outputs=output)
    model.compile(optimizer='adam', loss='mse')

    model.fit(Xtrain, (Y_train) , epochs=1000, batch_size=16 ,verbose=0)
    pred = model.predict(Xtest)

    error = (mean_squared_error(pred, Y_test)).round(3)
    Rsquare = (r2_score(pred, Y_test)).round(3)
    return error , Rsquare

In [23]:
def nnModel(X, Y1):
    losses = []
    R2es = []
    t = time.process_time()
    for i in range(100):
        X_train, X_test,Y_train, Y_test = train_test_split(X, Y1 , test_size = 0.20)
        loss, R2 = NN_MultiLayer(X_train, Y_train, 
                                 X_test, Y_test)
        losses.append(loss)
        R2es.append(R2)

    elapsed_time = time.process_time() - t  

    print('\nMSE for NN is: ', np.mean(np.array(losses))*100)
    print('R2 for NN is: ', np.mean(np.array(R2es))*100, ' %')
    print('time: ',round(elapsed_time/100, 6), '\n*********\n')

## NN performances for different outputs

In [24]:
OUTPUTS = ['Safety_factor', 'Oop_deform', 'Tot_contactEngy', 'Elast_strainEngy',
       'Edge_temp', 'Avr_frictForce', 'HeatRate', 'IntEngy', 'FricDissipRate']

for output in OUTPUTS:
    Y1, hyperparams = Hyperparam(output)
    nnModel(X, Y1)
    print('***********************************************************')

2022-03-10 18:10:53.984961: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-10 18:10:54.072705: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)



MSE for NN is:  6.563
R2 for NN is:  -49.45399999999999  %
time:  4.51733 
*********

***********************************************************

MSE for NN is:  1.338
R2 for NN is:  47.16199999999999  %
time:  4.66174 
*********

***********************************************************

MSE for NN is:  5.096
R2 for NN is:  28.978999999999992  %
time:  4.16395 
*********

***********************************************************

MSE for NN is:  2.6879999999999997
R2 for NN is:  51.61800000000001  %
time:  4.386087 
*********

***********************************************************

MSE for NN is:  2.0499999999999994
R2 for NN is:  51.38900000000001  %
time:  4.864952 
*********

***********************************************************

MSE for NN is:  3.2390000000000003
R2 for NN is:  46.763  %
time:  4.370266 
*********

***********************************************************

MSE for NN is:  2.154
R2 for NN is:  56.92700000000001  %
time:  4.552681 
*********

****